![image](ecomrag.jpg)

In [ ]:
#dependencies need to be installed
!pip install pypdf
!pip install jq
!pip install pinecone-client
!pip install pinecone_text
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install PyPDF2
!pip install chromadb
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install rank_bm25


## Importing necessary libraries

In [17]:
#importing the necesary libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_community.retrievers import PineconeHybridSearchRetriever
import pinecone

## Data extraction from PDF

In [3]:
pdf_loader = PyPDFLoader("/Users/shreehari/Documents/GenAi projects/ecommercerag/csvragdatas/Attachment 2.pdf")

## Data extraction from CSV

In [4]:
# csv_loader1 = CSVLoader(file_path='/Users/shreehari/Documents/GenAi projects/ecommercerag/csvragdatas/amazon_co-ecommerce_sample.csv')
csv_loader2 = CSVLoader(file_path='/Users/shreehari/Documents/GenAi projects/ecommercerag/csvragdatas/myntra_products_catalog.csv')


## Data extraction from JSON

In [5]:
#data extraction from json
json_loader = JSONLoader(
    file_path='/Users/shreehari/Documents/GenAi projects/ecommercerag/csvragdatas/amazon_uk_shoes_dataset.json',
    jq_schema='.[] | {title, price,brand}',
    text_content=False)


## Data extraction from websites

In [6]:
# web_loader = WebBaseLoader(["https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar","https://www.amazon.in/s?i=apparel&rh=n%3A1571271031%2Cn%3A1968024031%2Cn%3A1968120031%2Cn%3A94998646031&dc&fs=true&ds=v1%3AMGv3hbK1Hxx9Zu5q8x4tMmFfb5250gBqlYXAUHeQweY&qid=1721270273&rnid=1571271031&ref=sr_nr_n_2","https://www.amazon.in/s?i=apparel&rh=n%3A1571271031%2Cn%3A1968024031%2Cn%3A1968125031&dc&fs=true&ds=v1%3AOwW1wx%2BSvLHfZi%2FbtIGNZrZNJxq93%2Fk3dZimAShyP2E&qid=1721270610&rnid=1571271031&ref=sr_nr_n_3","https://www.amazon.in/s?i=apparel&rh=n%3A1571271031%2Cn%3A1968024031%2Cn%3A1968125031&dc&fs=true&page=2&qid=1721270623&rnid=1571271031&ref=sr_pg_2","https://sell.amazon.in/sell-online/faq"])
# data = web_loader.load()
# data
web_loader = WebBaseLoader(["https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar"])


# Due to Resource constraint only website and pdf are used!!

## Merging all the loaded Data

In [8]:
loader_all = MergedDataLoader(loaders=[web_loader,pdf_loader])
docs_all = loader_all.load()
docs_all

[Document(metadata={'source': 'https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar', 'title': 'Amazon.in', 'language': 'en-in'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAmazon.in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n.in\n\n\n\n\n\n\n\n\n\n                   Delivering to Coimbatore 641014\n                \n\n                   Update location\n                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMen's T-shirts, Polos & Shirts\n\n\nSelect the department you want to search in\n\nMen's T-shirts, Polos & Shirts\nAll Categories\nAlexa Skills\nAmazon Devices\nAmazon Fashion\nAmazon Fresh\nAmazon Pharmacy\nAppliances\nApps & Games\nAudible Audiobooks\nBaby\nBeauty\nBooks\nCar & Motorbike\nClothing & Accessories\nCollectibles\nComputers & Accessories\nDeals\nElectronics\nFurniture\nGarden & Outdoors\nGift Cards\nGrocery & Gourmet Food

## Converting the data into chunks

In [9]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs_all)
documents

[Document(metadata={'source': 'https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar', 'title': 'Amazon.in', 'language': 'en-in'}, page_content="Amazon.in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n.in\n\n\n\n\n\n\n\n\n\n                   Delivering to Coimbatore 641014\n                \n\n                   Update location\n                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMen's T-shirts, Polos & Shirts\n\n\nSelect the department you want to search in"),
 Document(metadata={'source': 'https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar', 'title': 'Amazon.in', 'language': 'en-in'}, page_content="Update location\n                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMen's T-shirts, Polos & Shirts\n\n\nSelect the department you want to search in\n\nMen's T-shirts, Polos & Shirts\nAll Categories\nAlexa Skills\nAmazon Devices\nAmazon Fashion\nAmazon Fresh\nA

# Configuring the Google-gemini pro model using API

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

## Embedding the documents using GooglegenAI model and store in Faiss Vector Store

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents, embedding=embeddings)
vector_store.save_local("faiss_index")


I0000 00:00:1721294630.472014 10380004 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1721294630.483684 10380004 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


# Creating Pinecone Serverless Index and connecting

In [12]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="3b643c4b-5264-49b9-bc2a-b9d4063e52dd")
pc.create_index(
    name="quickstart",
    dimension=768, # Replace with your model dimensions
    metric="dotproduct", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)



### Creating new embedding model for Pinecone Retriever and assigning the Index

In [13]:
embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
index = pc.Index("quickstart")

I0000 00:00:1721294872.764981 10380004 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


I0000 00:00:1721294890.663612 10389047 tcp_posix.cc:809] IOMGR endpoint shutdown


## Creating BM25Encoder to use PineconeHybridSearchRetriever

In [14]:
from pinecone_text.sparse import BM25Encoder

# use default tf-idf values
bm25_encoder = BM25Encoder().default()

I0000 00:00:1721294972.197953 10380004 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork


### Saving the encoded documents in json format

In [15]:
import json

# assume your documents are in a list called `documents`

# extract the text content from each document
text_documents = []
for doc in documents:
    text_documents.append(doc.page_content)

# fit the BM25 encoder to the text documents
bm25_encoder.fit(text_documents)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

I0000 00:00:1721295026.955165 10380004 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork


  0%|          | 0/65 [00:00<?, ?it/s]

# Creating Pinecone Hybrid Search Retriever

In [18]:
pc_retriever = PineconeHybridSearchRetriever(
    embeddings=embed, sparse_encoder=bm25_encoder, index=index
)

## Adding the extracted documents to the vector store

In [19]:
pc_retriever.add_texts(text_documents)

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
result = pc_retriever.invoke("best shirts under 1000")
result

[Document(page_content="+12Van HeusenMen's Solid Polo T Shirt  4.0 out of 5 stars 37,311  400+ bought in past month₹899₹899 M.R.P: ₹1,099₹1,099 (18% off) FREE Delivery by Amazon\nLymioCasual Shirt for Men|| Shirt for Men|| Men Stylish Shirt (Rib-Shirt)  3.9 out of 5 stars 1,075  500+ bought in past monthLimited time deal  ₹479₹479 M.R.P: ₹3,999₹3,999 (88% off)FREE Delivery by Amazon\nJockeyMen's Inner Regular Fit T Shirt  4.0 out of 5 stars 2,216  400+ bought in past month₹299₹299 M.R.P: ₹379₹379 (21% off)FREE Delivery by Amazon\nBest sellerin Men's Shirts  +22Dennis LingoMen's Solid Slim Fit Cotton Casual Shirt with Spread Collar & Full Sleeves (Also Available in Plus Size)  3.8 out of 5 stars 37,991  400+ bought in past month₹499₹499 M.R.P: ₹1,849₹1,849 (73% off) FREE Delivery by Amazon\n+5Allen SollyMen's Slim Fit Shirt  3.7 out of 5 stars 15,183  500+ bought in past month₹869₹869 M.R.P: ₹1,499₹1,499 (42% off) FREE Delivery by Amazon"),
 Document(page_content="Dixcy ScottOriginals M

# Creating Converstational Chain [Prompt Template + Query + LLM]

In [21]:
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context and with your own knowledge, make sure to provide all the details, if the answer is not in
    provided context then try to give a the correct answer with your knowledge but make sure you don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n
    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

I0000 00:00:1721295456.023431 10393760 tcp_posix.cc:809] IOMGR endpoint shutdown


## Creating Ensemble Retriever [Bm25 + Faiss + Pinecone Hybrid]

In [22]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create BM25 retriever
# bm25_retriever = BM25Retriever(docs=documents)
bm25_retriever = BM25Retriever.from_documents(
    documents
)

# Create FAISS retriever
faiss_retriever = new_db.as_retriever()

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever,pc_retriever], weights=[0.5, 0.5,0.5]
)

# Getting Relevant Documents based on query using Reciprocal Rank Fusion Algorithm

In [23]:
chain = get_conversational_chain()
query = "shirts under 1000"
docs = ensemble_retriever.invoke(query)
print(docs)

I0000 00:00:1721295535.526224 10380004 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721295535.535529 10380004 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


[Document(metadata={'source': 'https://www.amazon.in/s?rh=n%3A1968120031&fs=true&ref=lp_1968120031_sar', 'title': 'Amazon.in', 'language': 'en-in'}, page_content="+12Van HeusenMen's Solid Polo T Shirt  4.0 out of 5 stars 37,311  400+ bought in past month₹899₹899 M.R.P: ₹1,099₹1,099 (18% off) FREE Delivery by Amazon\nLymioCasual Shirt for Men|| Shirt for Men|| Men Stylish Shirt (Rib-Shirt)  3.9 out of 5 stars 1,075  500+ bought in past monthLimited time deal  ₹479₹479 M.R.P: ₹3,999₹3,999 (88% off)FREE Delivery by Amazon\nJockeyMen's Inner Regular Fit T Shirt  4.0 out of 5 stars 2,216  400+ bought in past month₹299₹299 M.R.P: ₹379₹379 (21% off)FREE Delivery by Amazon\nBest sellerin Men's Shirts  +22Dennis LingoMen's Solid Slim Fit Cotton Casual Shirt with Spread Collar & Full Sleeves (Also Available in Plus Size)  3.8 out of 5 stars 37,991  400+ bought in past month₹499₹499 M.R.P: ₹1,849₹1,849 (73% off) FREE Delivery by Amazon\n+5Allen SollyMen's Slim Fit Shirt  3.7 out of 5 stars 15,183

I0000 00:00:1721295781.222513 10393760 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721295781.222790 10393760 tcp_posix.cc:809] IOMGR endpoint shutdown


# Finally Getting the Output from the LLM

In [90]:

response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
print("Reply: ", response["output_text"])

Reply:  - LymioCasual Shirt for Men|| Shirt for Men|| Men Stylish Shirt (Rib-Shirt)  ₹479
- LEOTUDEMen's Regular Fit Half Sleeve Round Neck Dri-Fit Tshirt (Color Black)  ₹498
- JockeyMen's Inner Regular Fit T Shirt  ₹299
- VIP SupremeMen's Sleeveless Cotton Vest (Pack of 5)  ₹499
- Allen SollyMen's Slim Fit Shirt  ₹869
- Leriya Fashiont shirt for men| white t shirt for men| black drop shoulder t-shirt for men  ₹399
- Amazon Brand - SymbolMen's Solid Cotton Formal Shirt | Plain | Full Sleeve - Regular Fit (Available in Plus Size)  ₹549
- VIPSupreme Men's Cotton Vest (Pack of 3)  ₹297
- Amazon Brand - SymbolMen's Cotton T Shirt | Round Neck | Half Sleeve | Plain - Regular Fit (Available in Plus Size)  ₹299
- LEOTUDEMen Half Sleeve Oversized Tshirt, Round Neck Longline Drop Shoulder | Colorful Printed Combo Regular Fit T-Shirt (Pack Of 3)  ₹699
- Van HeusenMen's Solid T Shirt  ₹379
- Van HeusenMen's Solid Polo T Shirt  ₹899
- Lux CoziMen's Round Neck Sleeveless 100% Cotton Vest with Exten

I0000 00:00:1721281898.797194 10113488 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721281918.815147 10113514 tcp_posix.cc:809] IOMGR endpoint shutdown


# Due to lack of credits - I cant access Open AI model and Open Ai embeddings , so I have used open source Googleai embeddings and googlegemini pro llms. But i can also implement using Open Ai models